This a simple baseline using the LGBM algorithm. 

It builds upon :  https://www.kaggle.com/sishihara/riiid-lgbm-5cv-benchmark?rvi=1


In [ ]:
import riiideducation
import dask.dataframe as dd
import  pandas as pd
import numpy as np
env = riiideducation.make_env()

In [ ]:
train= pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                usecols=[1, 2, 3,7, 8, 9], dtype={'timestamp': 'int64', 'user_id': 'int64' ,'content_id': 'int16','answered_correctly':'int8', 'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'}
              )

In [ ]:
train.head()

In [ ]:
#Make a simple tree model and estimate score

In [ ]:
train.tail()

In [ ]:
train['prior_question_had_explanation'].fillna(0, inplace=True)
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('int8')

train['prior_question_elapsed_time'].fillna(train['prior_question_elapsed_time'].mean(), inplace=True)

In [ ]:
#arrange by timestamp

train = train.sort_values(['timestamp'], ascending=True)
train.head()

In [ ]:
train.tail()

In [ ]:
train.drop(['timestamp'], axis=1,   inplace=True)

In [ ]:
len(list(set(train.iloc[0:90000000,:]['user_id']).intersection(set(train.iloc[90000000:99000000,:]['user_id']))))

In [ ]:
results_c = train.iloc[0:90000000,:][['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

In [ ]:
results_u = train.iloc[0:90000000,:][['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']
results_u.head()

In [ ]:
X = train.iloc[90000000:99000000,:]


In [ ]:
X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")

In [ ]:
X=X[X.answered_correctly!= -1 ]
X=X.sort_values(['user_id'])


In [ ]:
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)
X.head()

In [ ]:
Y

In [ ]:
train.head()

In [ ]:
X = X[['answered_correctly_user', 'answered_correctly_content', 'sum',  'prior_question_had_explanation' ,'prior_question_elapsed_time']] 
X['answered_correctly_user'].fillna(0.5,  inplace=True)
X['answered_correctly_content'].fillna(0.5,  inplace=True)
X['sum'].fillna(0,  inplace=True)

In [ ]:
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.model_selection import train_test_split
Xt, Xv, Yt, Yv = train_test_split(X, Y, test_size =0.2, shuffle=False)

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'max_bin': 600,
    'learning_rate': 0.02,
    'num_leaves': 80
}


lgb_train = lgb.Dataset(Xt, Yt)
lgb_eval = lgb.Dataset(Xv, Yv, reference=lgb_train)

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

In [ ]:
y_pred = model.predict(Xv)
y_true = np.array(Yv)

In [ ]:
Y.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred)

In [ ]:
###Make sure it works on the test set

In [ ]:
test =  pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv' , dtype={'timestamp': 'int64', 'user_id': 'int64' ,'content_id': 'int16','answered_correctly':'int8', 'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'})
test.shape

In [ ]:
test = pd.merge(test, results_u, on=['user_id'],  how="left")
test = pd.merge(test, results_c, on=['content_id'],  how="left")

In [ ]:
test.shape

In [ ]:
test['prior_question_had_explanation'].fillna(0, inplace=True)
test['prior_question_had_explanation'] = test['prior_question_had_explanation'].astype('int8')
test['prior_question_elapsed_time'].fillna(train['prior_question_elapsed_time'].mean(), inplace=True)

In [ ]:
test[['answered_correctly_user', 'answered_correctly_content', 'sum', 'prior_question_had_explanation' ,'prior_question_elapsed_time']]
test['answered_correctly_user'].fillna(0.5,  inplace=True)
test['answered_correctly_content'].fillna(0.5,  inplace=True)
test['sum'].fillna(0,  inplace=True)

In [ ]:
y_pred = model.predict(test[['answered_correctly_user', 'answered_correctly_content', 'sum', 'prior_question_had_explanation' ,'prior_question_elapsed_time']])

In [ ]:
y_pred.shape

In [ ]:
test['answered_correctly'] = y_pred
test.head()

In [ ]:
#################

In [ ]:
#Make preds

results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

In [ ]:
results_c.head()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = pd.merge(test_df, results_u, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_c, on=['content_id'],  how="left")
    test_df['answered_correctly_user'].fillna(0.5, inplace=True)
    test_df['answered_correctly_content'].fillna(0.5, inplace=True)
    test_df['sum'].fillna(0, inplace=True)
    test_df['prior_question_had_explanation'].fillna(0, inplace=True)
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].astype('int8')
    test_df['prior_question_elapsed_time'].fillna(train['prior_question_elapsed_time'].mean(), inplace=True)
    test_df['answered_correctly'] =  model.predict(test_df[['answered_correctly_user', 'answered_correctly_content', 'sum','prior_question_elapsed_time','prior_question_had_explanation' ]])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])